In [ ]:
import time
import re
import pandas as pd

import selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from browser import getBrowserDriver

browser, waiter = getBrowserDriver()

# Pagina Inicial
el_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/a')))
el_extrajudicial.click()

el_serventias_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/ul/li[1]/a')))
el_serventias_extrajudicial.click()

# Criando dataframe
data = {
    'Denominacao': [""],
    'Situacao': [""],
    'Atribuicoes': [""],
    'Responsavel': [""],
    'Substituto': [""],
    'UF': [""],
    'Municipio': [""],
    'Telefone Principal': [""],
    'Telefone Secundario': [""],
    'Email': [""]
}

df = pd.DataFrame(data)

In [ ]:
def getEstados():
    el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
    el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

    return el_mapa_ufs_geral, el_mapa_ufs

def getMunicipios():
    el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
    el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

    return el_mapa_muns_geral, el_mapa_muns

def getDados():
    #Dados cartorio
    denominacao = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[1]/table/tbody/tr[2]/td[2]'))).text
    situacao = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[1]/table/tbody/tr[5]/td[2]'))).text

    #Atribuições
    atribuicoes = ""
    element_temp = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[2]/table/tbody')))
    elements_temp = element_temp.find_elements(By.TAG_NAME, 'td')
    for c in range(0, len(elements_temp)):
            if(c % 2 == 1 and c != 1):
                atribuicoes += f"{elements_temp[c].text[1:]} ;"

    #Responsaveis
    responsavel = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[3]/table/tbody/tr[1]/td[2]'))).text
    responsavel = re.sub(r"\n.*", "", responsavel)
    substituto = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[3]/table/tbody/tr[3]/td[2]'))).text
    substituto = re.sub(r"\n.*", "", substituto)

    #Localização
    uf = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[1]/td[2]'))).text
    municipio = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[2]/td[2]'))).text
    telefone_principal = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[6]/td[2]'))).text
    telefone_secundario = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[7]/td[2]'))).text

    try:
        email = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[8]/td[2]'))).text
    except:
        email = "Null"
    
    element_temp = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody')))
    element_temp = element_temp.find_elements(By.TAG_NAME, 'tr')

    if(len(element_temp) == 7):
        email = telefone_secundario
        telefone_secundario = "Null"
    
    nova_linha = {
        'Denominacao': denominacao,
        'Situacao': situacao,
        'Atribuicoes': atribuicoes,
        'Responsavel': responsavel,
        'Substituto': substituto,
        'UF': uf,
        'Municipio': municipio,
        'Telefone Principal': telefone_principal,
        'Telefone Secundario': telefone_secundario,
        'Email': email
    }

    nova_linha_df = pd.DataFrame([nova_linha])  # Criar um novo DataFrame com a nova linha

    return nova_linha_df

In [ ]:
# Pagina Mapa
i = 0
# Selecionando elementos
el_mapa_ufs_geral, el_mapa_ufs = getEstados()

# Selecionando estados
for uf in range(1, len(el_mapa_ufs)):
    el_mapa_ufs_geral, el_mapa_ufs = getEstados()
    el_mapa_ufs[uf].click()
    browser.implicitly_wait(2)

    # Selecionando municipios
    el_mapa_muns_geral, el_mapa_muns = getMunicipios()

    for num in range(1, len(el_mapa_muns)):
        el_mapa_muns[num].click()

        el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
        el_pesquisar.click()  

        # Pagina Cartorios 

        # Selecionando dados
        el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
        el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')
        
        for info in range(0, len(el_infos)):
            el_infos[info].click()
            try:
                el_infos[info].click()
            except:
                ...

            browser.implicitly_wait(3)
            nova_linha_df = getDados()
            df = pd.concat([df, nova_linha_df], ignore_index=True)  # Concatenar os DataFrames

            browser.back()
            browser.implicitly_wait(1)
            
            el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
            el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')
            i+= 1

        if(i == 100):
            browser.close()
            break
            
        # Resetando para conseguir clicar novamente
        browser.back()
        browser.back()
        browser.implicitly_wait(3)
        el_mapa_ufs_geral, el_mapa_ufs = getEstados()
        el_mapa_ufs[uf].click()
        el_mapa_muns_geral, el_mapa_muns = getMunicipios()

    if(i == 100):
            browser.close()
            break

In [ ]:
df = df.drop(index = 0)
df

In [ ]:
el_infos[info].click()

In [ ]:
# # Pagina Mapa

# # Selecionando estados
# el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
# el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

# el_mapa_ufs[-10].click()

# # Selecionando municipios
# el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
# el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

# el_mapa_muns[19].click()

# # Pesquisar
# el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
# el_pesquisar.click()

In [ ]:
# # Pagina Cartorios 

# # Selecionando dados
# el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
# el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

# for info in range(0, len(el_infos)):
#     if((info + 1) % 2 == 0):
#         el_infos[info].click()
#         el_infos[info].click()
#     else:
#         el_infos[info].click()

#     time.sleep(2)
#     nova_linha_df = getDados()
#     df = pd.concat([df, nova_linha_df], ignore_index=True)  # Concatenar os DataFrames

#     time.sleep(1)
#     browser.back()
#     time.sleep(2)
    
#     el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
#     el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

In [ ]:
# # Pagina Cartorios 

# # Selecionando dados
# el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
# el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

# element_temp = el_infos_geral.find_elements(By.TAG_NAME, 'tr')
# decisao = element_temp[0].get_attribute('class')

# if(decisao != "processo odd"):
#     print("revert")